In [5]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
from faker import Faker
from typing import List

In [6]:
client = QdrantClient(host="localhost", port=6333)

In [7]:
my_collection = "first_collection"
client.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)

True

In [8]:
data = np.random.uniform(low=0.0, high=1.0, size=(1_000, 100))
index: List = list(range(1_000))
index[-10:]

[990, 991, 992, 993, 994, 995, 996, 997, 998, 999]

In [9]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist(),
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
client.retrieve(
    collection_name=my_collection,
    ids=[10,14,500],
)

[Record(id=500, payload={}, vector=None, shard_key=None),
 Record(id=14, payload={}, vector=None, shard_key=None),
 Record(id=10, payload={}, vector=None, shard_key=None)]

In [11]:
fakerInstance = Faker()

payload: List[models.Payload] = []

for i in range(1_000):
    payload.append({
        "artist": fakerInstance.name(),
        "song": " ".join(fakerInstance.words()),
        "url_song": fakerInstance.url(),
        "year": fakerInstance.year(),
        "country": fakerInstance.country()
    })
    
payload[:2]

[{'artist': 'Daniel Hess',
  'song': 'poor positive who',
  'url_song': 'https://hill-harmon.com/',
  'year': '1975',
  'country': 'Kuwait'},
 {'artist': 'Nathan Brown',
  'song': 'record give personal',
  'url_song': 'https://www.church-walker.com/',
  'year': '2022',
  'country': 'Vietnam'}]

In [12]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist(),
        payloads=payload
    )
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

## Semantic Search

In [13]:
living_la_vida_loca = np.random.uniform(low=-1.0, high=1.0, size=(100)).tolist()
living_la_vida_loca[:5]

[0.5934405285210862,
 -0.031250302953845344,
 -0.4395631275418581,
 -0.5910097345329408,
 -0.5351707327860356]

In [14]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    limit=5
)

[ScoredPoint(id=533, version=1, score=0.053160086, payload={'artist': 'Luis Robinson', 'country': 'Antigua and Barbuda', 'song': 'return write front', 'url_song': 'http://reed.com/', 'year': '2006'}, vector=None, shard_key=None),
 ScoredPoint(id=37, version=1, score=0.049821205, payload={'artist': 'Gina Cruz', 'country': 'Iraq', 'song': 'always might collection', 'url_song': 'http://moore-gordon.com/', 'year': '1977'}, vector=None, shard_key=None),
 ScoredPoint(id=969, version=1, score=0.049208406, payload={'artist': 'Haley Jones', 'country': 'Kuwait', 'song': 'fund past move', 'url_song': 'http://archer.com/', 'year': '1994'}, vector=None, shard_key=None),
 ScoredPoint(id=858, version=1, score=0.036787193, payload={'artist': 'Paul Vasquez', 'country': 'Palau', 'song': 'prepare figure day', 'url_song': 'http://taylor.info/', 'year': '1981'}, vector=None, shard_key=None),
 ScoredPoint(id=437, version=1, score=0.031494234, payload={'artist': 'Matthew Mcdonald', 'country': 'Senegal', 'son

### French songs only

In [15]:
french_songs = models.Filter(
    must=[
        models.FieldCondition(
            key="country",
            match=models.MatchValue(value="France")
        )
    ]
)
french_songs

Filter(should=None, must=[FieldCondition(key='country', match=MatchValue(value='France'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)], must_not=None)

In [16]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    query_filter=french_songs,
    limit=5,
)

[ScoredPoint(id=350, version=1, score=-0.062317606, payload={'artist': 'Brent Phillips', 'country': 'France', 'song': 'open message available', 'url_song': 'http://www.jensen.com/', 'year': '1995'}, vector=None, shard_key=None),
 ScoredPoint(id=707, version=1, score=-0.11642556, payload={'artist': 'Thomas Gonzales', 'country': 'France', 'song': 'notice American itself', 'url_song': 'http://faulkner-reeves.info/', 'year': '1983'}, vector=None, shard_key=None),
 ScoredPoint(id=836, version=1, score=-0.18270856, payload={'artist': 'Mrs. Eileen Medina', 'country': 'France', 'song': 'than will avoid', 'url_song': 'https://montgomery.biz/', 'year': '2019'}, vector=None, shard_key=None)]

## Recommendation

In [20]:
client.recommend(
    collection_name=my_collection,
    positive=[17,19,944],
    negative=[100, 444],
    query_filter=french_songs,
    limit=5
)

[ScoredPoint(id=836, version=1, score=0.737018, payload={'artist': 'Mrs. Eileen Medina', 'country': 'France', 'song': 'than will avoid', 'url_song': 'https://montgomery.biz/', 'year': '2019'}, vector=None, shard_key=None),
 ScoredPoint(id=707, version=1, score=0.7127339, payload={'artist': 'Thomas Gonzales', 'country': 'France', 'song': 'notice American itself', 'url_song': 'http://faulkner-reeves.info/', 'year': '1983'}, vector=None, shard_key=None),
 ScoredPoint(id=350, version=1, score=0.69959104, payload={'artist': 'Brent Phillips', 'country': 'France', 'song': 'open message available', 'url_song': 'http://www.jensen.com/', 'year': '1995'}, vector=None, shard_key=None)]